In [1]:
import pandas as pd
import string
import datetime
from datetime import datetime
from datetime import timedelta
import numpy as np
import os

from random import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
# some functions

def change_player(player):
    player=player.split(' ')
    player=' '.join(player[1:])+'_'+player[0][0]+'.'
    return player

def change_player_slug(player_slug):
    player_slug=player_slug.split('-')
    player_slug=''.join(player_slug[1:])+'_'+player_slug[0][0]+'.'
    return player_slug

In [3]:
# read in all match data since 1991
match=pd.DataFrame()
for k in range (1991,2021): 
    df=pd.read_csv('..\\raw_data\\atp_matches_'+str(k)+'.csv')
    match=pd.concat([match,df])
    
# create a new dataframe with just data on the match winners
match_winner=match[['tourney_date',
       'winner_name', 'winner_age','winner_hand', 'winner_ht', 'winner_ioc',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced']]

match_loser=match[[
       'tourney_date','loser_name','loser_age','loser_hand', 'loser_ht','loser_ioc',
       'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon',
       'l_SvGms', 'l_bpSaved', 'l_bpFaced']]

match_winner['outcome'] = "win"
match_winner['outcome'] = "loss"

match_winner = match_winner.rename(columns={"winner_name": "player_name", "winner_age": "player_age", "winner_hand": "player_hand"
                            , "winner_ht": "player_ht", "winner_ioc": "player_loc", "w_ace": "ace"
                            , "w_df": "df", "w_svpt": "svpt", "w_1stIn": "1stIn"
                            , "w_1stWon": "1stWon", "w_2ndWon": "2ndWon", "w_SvGms": "SvGms"
                            , "w_bpSaved": "bpSaved", "w_bpFaced": "bpFaced"})

match_loser = match_loser.rename(columns={"loser_name": "player_name", "loser_age": "player_age", "loser_hand": "player_hand"
                            , "loser_ht": "player_ht", "loser_ioc": "player_loc", "l_ace": "ace"
                            , "l_df": "df", "l_svpt": "svpt", "l_1stIn": "1stIn"
                            , "l_1stWon": "1stWon", "l_2ndWon": "2ndWon", "l_SvGms": "SvGms"
                            , "l_bpSaved": "bpSaved", "l_bpFaced": "bpFaced"})

match_players = pd.concat([match_winner, match_loser])
# add a column for match year
match_players['year'] = match_players['tourney_date'].apply(lambda row: str(row)[0:4])

# remove all matches with no statistics
match_players = match_players[~match_players['ace'].isnull()]
# and players with no age who only appear in the dataset once
match_players = match_players[~match_players['player_age'].isnull()]
# and players with no hand--there aren't that many
match_players = match_players[~match_players['player_hand'].isnull()]
# players with no height get the median
match_players=match_players.fillna(match_players[['player_ht']].median())

In [4]:
# Group by player and year so we have yearly stats for each players
player_stats = match_players.groupby(['player_name','year','player_hand','player_ht','player_loc'], as_index = False)['ace','df','svpt','1stIn','1stWon','2ndWon','SvGms','bpSaved','bpFaced'].sum()
# just use a player's average age for the year
player_avg_age = match_players.groupby(['player_name', 'year'], as_index = False)['player_age'].mean()
# join
player_stats = pd.merge(player_stats,player_avg_age, how = 'left', on = ['player_name','year'])



In [5]:
# Calculate rate stats
player_stats['player_name'] = player_stats['player_name'].apply(lambda x:change_player(x)).str.lower()
player_stats['percentage_aces'] = player_stats['ace'] / player_stats['svpt']
player_stats['percentage_df'] = player_stats['df'] / player_stats['svpt']
player_stats['percentage_1stwon'] = player_stats['1stWon'] / player_stats['1stIn']
player_stats['percentage_bpsaved'] = player_stats['bpSaved'] / player_stats['bpFaced']
player_stats['percentage_1stsv'] = player_stats['1stIn'] / player_stats['svpt']
player_stats.head()

,player_name,year,player_hand,player_ht,player_loc,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,player_age,percentage_aces,percentage_df,percentage_1stwon,percentage_bpsaved,percentage_1stsv
0,krickstein_a.,1991,R,183.0,USA,205.0,108.0,3883.0,2285.0,1591.0,824.0,586.0,222.0,358.0,23.909472,0.052794,0.027814,0.696280,0.620112,0.588463
1,krickstein_a.,1992,R,183.0,USA,170.0,92.0,3469.0,1960.0,1368.0,792.0,549.0,174.0,302.0,24.694402,0.049005,0.026521,0.697959,0.576159,0.565004
2,krickstein_a.,1993,R,183.0,USA,205.0,96.0,3759.0,2192.0,1510.0,788.0,559.0,245.0,390.0,25.896601,0.054536,0.025539,0.688869,0.628205,0.583134
3,krickstein_a.,1994,R,183.0,USA,236.0,127.0,4593.0,2714.0,1867.0,967.0,690.0,201.0,358.0,26.812918,0.051383,0.027651,0.687915,0.561453,0.590899
4,krickstein_a.,1995,R,183.0,USA,166.0,96.0,3324.0,1968.0,1315.0,641.0,469.0,216.0,363.0,27.774904,0.049940,0.028881,0.668191,0.595041,0.592058


In [6]:
# Next add in data from separate file:
# birthdate, years pro, weight, handedness, backhand style
players=pd.read_csv('player_overviews_unindexed_csv.csv')
players['player_name']=players['player_slug'].apply(lambda x:change_player_slug(x))
players=players[['birthdate','turned_pro','weight_kg','handedness','backhand','player_name', 'player_id']]
players = pd.merge(player_stats, players, on = 'player_name', how = "left")
# missing a lot of data on backhand and handedness

# add a years pro column
players['years_pro'] = players['year'].apply(lambda x:int(x)) - players['turned_pro']
players.head()

,player_name,year,player_hand,player_ht,player_loc,ace,df,svpt,1stIn,1stWon,...,percentage_1stwon,percentage_bpsaved,percentage_1stsv,birthdate,turned_pro,weight_kg,handedness,backhand,player_id,years_pro
0,krickstein_a.,1991,R,183.0,USA,205.0,108.0,3883.0,2285.0,1591.0,...,0.696280,0.620112,0.588463,1967.08.02,1983.0,73.0,NaN,NaN,k023,8.0
1,krickstein_a.,1992,R,183.0,USA,170.0,92.0,3469.0,1960.0,1368.0,...,0.697959,0.576159,0.565004,1967.08.02,1983.0,73.0,NaN,NaN,k023,9.0
2,krickstein_a.,1993,R,183.0,USA,205.0,96.0,3759.0,2192.0,1510.0,...,0.688869,0.628205,0.583134,1967.08.02,1983.0,73.0,NaN,NaN,k023,10.0
3,krickstein_a.,1994,R,183.0,USA,236.0,127.0,4593.0,2714.0,1867.0,...,0.687915,0.561453,0.590899,1967.08.02,1983.0,73.0,NaN,NaN,k023,11.0
4,krickstein_a.,1995,R,183.0,USA,166.0,96.0,3324.0,1968.0,1315.0,...,0.668191,0.595041,0.592058,1967.08.02,1983.0,73.0,NaN,NaN,k023,12.0


In [7]:
players.to_csv("processed_data\\player_profiles_clean.csv")